### Find Word Images And Copy

In [36]:
import os
import sys
import pandas as pd
import numpy as np
import re
import glob
from pathlib import Path
import shutil
from os.path import isfile, join

In [37]:
# language pair
lang_folder = "Turkish"  # Arabic, English, French, German, Turkish, Spanish, Portuguese, Dutch, Italian ==> target language for learner
#lang_pair = "English"  # Arabic, English, French, German, Turkish, Spanish, Portuguese, Dutch, Italian ==> native language

file_ext = 1000

# image result parameter
select_image_num = 1  # 10
take_image_sample_num = 50  # 50

In [38]:
word_lemma_data_path = f"/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/{lang_folder.capitalize()}/\
Lemma Stem POS/Result/3-2-Word In Visual Genome Merge"

word_lemma_data_path2 = f"/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/{lang_folder.capitalize()}/\
Lemma Stem POS/Result/3-3-Find Word Images Mask Fade"

path = f"/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/{lang_folder.capitalize()}/\
Lemma Stem POS/Result/3-4-Find Word Images And Copy"

#Path(path).mkdir(parents=True, exist_ok=True)

In [39]:
def word_group_dataframe(df, search_list, target_column, sample_num):
    '''
    word_group_dataframe(df_youtube_sentence, search_list, "sentence", 6)\n
    df_youtube_sentence is dataframe and "sentence" is its column for external searching_list
    ''' 
    df_search_result = pd.DataFrame()
    for j in search_list:
        df_select = df[df[f"{target_column}"].str.contains(fr"(?:\s|^){j}(?:\s|$)", na=False, regex=True)]
        #df_select.sort_values(f"{target_column}",key=lambda x:x.str.len(), inplace=True).head(sample_num)
        df_select = df_select.sort_values(f"{target_column}",key=lambda x:x.str.len()).head(sample_num)               
        df_select.insert(0,"search_string",j)
        df_search_result = pd.concat([df_search_result,df_select], axis=0)
    df_search_result.reset_index(inplace=True, drop=True)
    
    return df_search_result

In [40]:
def word_group_dataframe_all(df, search_list, target_column):
    '''
    word_group_dataframe(df_youtube_sentence, search_list, "sentence", 6)\n
    df_youtube_sentence is dataframe and "sentence" is its column for external searching_list
    ''' 
    df_search_result = pd.DataFrame()
    for j in search_list:
        df_select = df[df[f"{target_column}"].str.contains(fr"(?:\s|^){j}(?:\s|$)", na=False, regex=True)]
        #df_select.sort_values(f"{target_column}",key=lambda x:x.str.len(), inplace=True)
        df_select = df_select.sort_values(f"{target_column}",key=lambda x:x.str.len())
        df_select.insert(0,"search_string",j)
        df_search_result = pd.concat([df_search_result,df_select], axis=0)        
    df_search_result.reset_index(inplace=True, drop=True)
    
    return df_search_result

In [41]:
def take_dataframe_word_sample_from_sorting(df_source, word_list, word_source_column, sort_target_column, sort_ascending=True, sample_num=50):
    '''take_dataframe_word_sample_from_sorting(df_source, word_list, word_source_column, sort_target_column, sort_ascending=True, sample_num=50)\n
    df_source is a dataframe and word_list is equal in word_source_column. Then sort_target_column is sorting according to sort_ascending condition.\n
    Finally, taking sample_num each word_list values.\n 
    ex.\n
    take_dataframe_word_sample_from_sorting(df_genome_word_lemma_concat, word_list, "word", "search_text", sort_ascending=True, sample_num=50)
    '''
    df_search_result = pd.DataFrame()
    for word in word_list:
        df_select = df_source[df_source[f"{word_source_column}"] == word]
        df_select = df_select.sort_values(f"{sort_target_column}",key=lambda x:x.str.len(), ascending=sort_ascending).head(sample_num)
        df_search_result = pd.concat([df_search_result,df_select], axis=0)
    
    df_search_result.reset_index(inplace=True, drop=True)
    
    return df_search_result

In [42]:
#def create_word_folder_and_copy_image(df_source, word_list, word_source_column, image_id_column, image_folder_path, output_path_folder):
#    '''create_word_folder_and_copy_image(df_source, word_list, word_source_column, image_id_column, image_folder_path, output_path_folder)\n
#    df_source is a dataframe and word_list is equal in word_source_column. Then word image id search in image_id_column and image copy to\n
#    output_path_folder from image_folder_path.\n
#    ex.\n
#    image_path = f"/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/{lang_folder.capitalize()}/Lemma Stem POS/Data/Visual Genome/images"\n
#    output_path_folder = "/home/kurubal/Downloads/temp folder"\n
#    create_word_folder_and_copy_image(df_sample_result, word_list, "word", "image_id", image_path, output_path_folder)
#    '''
#    df_search_result = pd.DataFrame()
#    for word in word_list:
#        path = f"{output_path_folder}/{word}"
#        Path(path).mkdir(parents=True, exist_ok=True)        
#        df_select = df_source[df_source[f"{word_source_column}"] == word]
#        for image_id in df_select[f"{image_id_column}"]:
#            image_file = glob.glob(f"{image_folder_path}/*/{image_id}.jpg")
#            for l in image_file:
#                source = l # source directory
#                destination = path
#                shutil.copy2(source, destination)

In [43]:
def create_word_folder_and_copy_image(df_source, word_list, word_source_column, image_id_column, opt_column, image_folder_path, mask_folder_path, output_path_folder):
    '''create_word_folder_and_copy_image(df_source, word_list, word_source_column, image_id_column, image_folder_path, mask_folder_path, output_path_folder)\n
    df_source is a dataframe and word_list is equal in word_source_column. Then word image id search in image_id_column and image copy to\n
    output_path_folder from image_folder_path. opt_column used for additional features\n
    ex.\n
    image_path = f"/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/{lang_folder.capitalize()}/Lemma Stem POS/Data/Visual Genome/images"\n
    output_path_folder = "/home/kurubal/Downloads/temp folder"\n
    create_word_folder_and_copy_image(df_sample_result, word_list, "word", "image_id", "num", image_path, mask_image_path output_path_folder)
    '''
    df_search_result = pd.DataFrame()
    for word in word_list:
        path = f"{output_path_folder}/{word}"
        Path(path).mkdir(parents=True, exist_ok=True)        
        df_select = df_source[df_source[f"{word_source_column}"] == word]
        imageid_list = list(df_select[f"{image_id_column}"])
        opt_column_list = list(df_select[f"{opt_column}"])
        data_list_zip = zip(imageid_list, opt_column_list)
        for image_id, opt in data_list_zip:
            opt_num = int(opt)
            
            #if isfile(f"{image_folder_path}/VG_Mask_Blur/{image_id}_{opt_num}_mask_blur.jpg"):  # ?????                
            #    image_file = glob.glob(f"{image_folder_path}/*/{image_id}_{opt_num}_mask_blur.jpg")

            if isfile(f"{mask_folder_path}/{image_id}_{opt_num}_mask_blur.jpg"):  # ?????                
                image_file = glob.glob(f"{mask_folder_path}/{image_id}_{opt_num}_mask_blur.jpg")
                for l in image_file:
                    source = l # source directory
                    destination = path
                    shutil.copy2(source, destination)
            else:
                image_file = glob.glob(f"{image_folder_path}/*/{image_id}.jpg")
                for l in image_file:
                    source = l # source directory
                    destination = path
                    shutil.copy2(source, destination)

In [44]:
image_path = f"/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/{lang_folder.capitalize()}/Lemma Stem POS/Data/Visual Genome/images"
mask_image_path = f"/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/{lang_folder.capitalize()}/Lemma Stem POS/Data/Visual Genome/images/VG_Mask_Blur"

In [45]:
image_folder_list = glob.glob(f"{image_path}/*")
image_folder_list

['/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/Turkish/Lemma Stem POS/Data/Visual Genome/images/VG_100K',
 '/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/Turkish/Lemma Stem POS/Data/Visual Genome/images/VG_100K_2',
 '/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/Turkish/Lemma Stem POS/Data/Visual Genome/images/VG_Mask_Blur']

#### Visual Genome Word Lemma Translate Data

In [ ]:
Pos_Tag = "NOUN"

In [ ]:
output_path_folder = f"/home/kurubal/Downloads/Image_Select/{Pos_Tag}"

In [47]:
df_genome_word_lemma_concat = pd.read_csv(f"{word_lemma_data_path}/Visual_Genome_{file_ext}_Word_Lemma_Search_Result.csv")
df_genome_word_lemma_concat

,POS1,POS2,word,lemma.spacy,stem,word_en_translate,lemma_en_translate,frequency,search_text,image_id
0,NUM,NaN,bir,bir,bir,a,a,18835735,a,2390994
1,NUM,NaN,bir,bir,bir,a,a,18835735,a,2348965
2,NUM,NaN,bir,bir,bir,a,a,18835735,a,2349861
3,NUM,NaN,bir,bir,bir,a,a,18835735,a,2349866
4,NUM,NaN,bir,bir,bir,a,a,18835735,a,2349935
...,...,...,...,...,...,...,...,...,...,...
176316,VERB,NaN,çekilin,çek,çek,withdraw,check,69201,airport check in kiosks,2317616
176317,VERB,NaN,çekilin,çek,çek,withdraw,check,69201,red check of tablecloth,2400604
176318,VERB,NaN,çekilin,çek,çek,withdraw,check,69201,a check is on the table,2386272
176319,VERB,NaN,çekilin,çek,çek,withdraw,check,69201,cleats with white check,2371210


In [ ]:
df_genome_word_lemma_concat = df_genome_word_lemma_concat[df_genome_word_lemma_concat["POS1"] == Pos_Tag]
df_genome_word_lemma_concat

In [48]:
df_genome_word_lemma_concat_coor = pd.read_csv(f"{word_lemma_data_path2}/Visual_Genome_{file_ext}_Word_Lemma_Coordinate_Search_Result.csv")
df_genome_word_lemma_concat_coor = df_genome_word_lemma_concat_coor.drop(["height","width","x_koor","y_koor"], axis=1)
df_genome_word_lemma_concat_coor

,num,word,lemma.spacy,stem,word_en_translate,image_id
0,1,bir,bir,bir,a,2348965
1,2,bir,bir,bir,a,2348965
2,4,bir,bir,bir,a,2349866
3,7,bir,bir,bir,a,2350230
4,10,bir,bir,bir,a,2350444
...,...,...,...,...,...,...
66751,197121,çekilin,çek,çek,withdraw,2365274
66752,197123,çekilin,çek,çek,withdraw,2387284
66753,197124,çekilin,çek,çek,withdraw,2400843
66754,197128,çekilin,çek,çek,withdraw,2333148


In [49]:
df_genome_word_lemma_concat_select = df_genome_word_lemma_concat.groupby("word")["image_id"].apply(lambda x: x.count()>=select_image_num).reset_index()
df_genome_word_lemma_concat_select

,word,image_id
0,acaba,True
1,acele,True
2,acı,True
3,adam,True
4,adama,True
...,...,...
978,şimdiye,True
979,şu,True
980,şuna,True
981,şunu,True


In [50]:
word_select_list = df_genome_word_lemma_concat_select[df_genome_word_lemma_concat_select["image_id"] == True]["word"].to_list()
#word_select_list

In [51]:
df_genome_word_lemma_concat_select_result = df_genome_word_lemma_concat[df_genome_word_lemma_concat["word"].isin(word_select_list)]
df_genome_word_lemma_concat_select_result

,POS1,POS2,word,lemma.spacy,stem,word_en_translate,lemma_en_translate,frequency,search_text,image_id
0,NUM,NaN,bir,bir,bir,a,a,18835735,a,2390994
1,NUM,NaN,bir,bir,bir,a,a,18835735,a,2348965
2,NUM,NaN,bir,bir,bir,a,a,18835735,a,2349861
3,NUM,NaN,bir,bir,bir,a,a,18835735,a,2349866
4,NUM,NaN,bir,bir,bir,a,a,18835735,a,2349935
...,...,...,...,...,...,...,...,...,...,...
176316,VERB,NaN,çekilin,çek,çek,withdraw,check,69201,airport check in kiosks,2317616
176317,VERB,NaN,çekilin,çek,çek,withdraw,check,69201,red check of tablecloth,2400604
176318,VERB,NaN,çekilin,çek,çek,withdraw,check,69201,a check is on the table,2386272
176319,VERB,NaN,çekilin,çek,çek,withdraw,check,69201,cleats with white check,2371210


In [52]:
df_genome_word_lemma_concat_select_result_coor_merge = pd.merge(df_genome_word_lemma_concat, df_genome_word_lemma_concat_coor,how="left", on=["word","image_id","word_en_translate","lemma.spacy","stem"])
#df_genome_word_lemma_concat_select_result_coor_merge = pd.merge(df_genome_word_lemma_concat, df_genome_word_lemma_concat_coor,how="left", on=["word","image_id"])
df_genome_word_lemma_concat_select_result_coor_merge = df_genome_word_lemma_concat_select_result_coor_merge.drop_duplicates()
df_genome_word_lemma_concat_select_result_coor_merge["num"] = df_genome_word_lemma_concat_select_result_coor_merge["num"].fillna(0)
df_genome_word_lemma_concat_select_result_coor_merge

,POS1,POS2,word,lemma.spacy,stem,word_en_translate,lemma_en_translate,frequency,search_text,image_id,num
0,NUM,NaN,bir,bir,bir,a,a,18835735,a,2390994,0.0
1,NUM,NaN,bir,bir,bir,a,a,18835735,a,2348965,1.0
2,NUM,NaN,bir,bir,bir,a,a,18835735,a,2348965,2.0
3,NUM,NaN,bir,bir,bir,a,a,18835735,a,2349861,0.0
4,NUM,NaN,bir,bir,bir,a,a,18835735,a,2349866,4.0
...,...,...,...,...,...,...,...,...,...,...,...
196273,VERB,NaN,çekilin,çek,çek,withdraw,check,69201,a check is on the table,2386272,197131.0
196274,VERB,NaN,çekilin,çek,çek,withdraw,check,69201,cleats with white check,2371210,0.0
196275,VERB,NaN,çekilin,çek,çek,withdraw,check,69201,check number on receipt,2404231,196965.0
196276,VERB,NaN,çekilin,çek,çek,withdraw,check,69201,check number on receipt,2404231,197009.0


In [53]:
df_genome_word_lemma_concat_select_result_coor_merge["word"].nunique()

983

In [54]:
word_list = list(set(df_genome_word_lemma_concat_select_result_coor_merge["word"]))

In [55]:
df_genome_sample_result = take_dataframe_word_sample_from_sorting(df_genome_word_lemma_concat_select_result_coor_merge, word_list, "word", 
                                                                    "search_text", sort_ascending=True, sample_num=take_image_sample_num)
df_genome_sample_result

,POS1,POS2,word,lemma.spacy,stem,word_en_translate,lemma_en_translate,frequency,search_text,image_id,num
0,ADV,Q,niye,niye,niye,why,why,292685,daddy why,2347954,0.0
1,ADV,Q,niye,niye,niye,why,why,292685,"white , why",2345124,10482.0
2,ADV,Q,niye,niye,niye,why,why,292685,why do this,2389512,0.0
3,ADV,Q,niye,niye,niye,why,why,292685,why is blue,2326509,0.0
4,ADV,Q,niye,niye,niye,why,why,292685,why is there,2355609,0.0
...,...,...,...,...,...,...,...,...,...,...,...
44768,NOUN,NEG,hayır,hayır,hayır,no,no,3406992,no,2398486,0.0
44769,NOUN,NEG,hayır,hayır,hayır,no,no,3406992,no,2413300,0.0
44770,NOUN,NEG,hayır,hayır,hayır,no,no,3406992,no,2384206,6138.0
44771,NOUN,NEG,hayır,hayır,hayır,no,no,3406992,no,2388737,6136.0


In [56]:
df_genome_sample_result["word"].nunique()

983

In [57]:
#df_genome_sample_result.to_excel("image_text.xlsx", index=False)

In [58]:
create_word_folder_and_copy_image(df_genome_sample_result, word_list, "word", "image_id", "num", image_path, mask_image_path ,output_path_folder)

In [33]:
for word in word_list:
    path = f"{output_path_folder}/ALL_{Pos_Tag}_FOLDER/{word}"
    Path(path).mkdir(parents=True, exist_ok=True)  

#### Copy Move And Delete

In [46]:
output_file = glob.glob(f"")
output_file

['Visual_Genome_Question_Answers_Word_Result2.csv',
 'Visual_Genome_Question_Answers_Lemma_Result2.csv']

In [47]:
for l in output_file:
    source = l # source directory
    destination = path
    shutil.copy2(source, destination)

In [48]:
for j in output_file:
    try:
        os.remove(j)
    except:
        pass

#### Temp

In [19]:
df_test = pd.read_excel("image_text.xlsx")
df_test

,POS1,POS2,word,lemma.spacy,stem,word_en_translate,lemma_en_translate,frequency,search_text,image_id
0,ADJ,NaN,üstünde,üst,üst,above,top,86801,top,2371537
1,ADJ,NaN,üstünde,üst,üst,above,top,86801,top,2328281
2,ADJ,NaN,üstünde,üst,üst,above,top,86801,top,2328272
3,ADJ,NaN,üstünde,üst,üst,above,top,86801,top,2328221
4,ADJ,NaN,üstünde,üst,üst,above,top,86801,top,2328063
...,...,...,...,...,...,...,...,...,...,...
5395,NOUN,NaN,konusunda,konu,konu,about,subject,167046,about half,2390601
5396,NOUN,NaN,konusunda,konu,konu,about,subject,167046,about half,2402492
5397,NOUN,NaN,konusunda,konu,konu,about,subject,167046,no subject,2318736
5398,NOUN,NaN,konusunda,konu,konu,about,subject,167046,no subject,2356446
